<div style="position:relative; height:160px;">
<img src="logoDECATUFS.png" width="500" height="500" style="position:absolute; left:-8px;">
</div>

<div style="text-align:center;font-size:17px; white-space:pre;margin-top:-35px">
        <h3>             Modelos Lineares Generalizados</h3>
        Disciplina ofertada pelo DECAT/UFS
        Código: ESTAT0092
        Nível: Graduação
        Carga horária: 60h
        Período: 2020.2
        Professor responsável e ministrante: Luiz Henrique Dore
        <h3>            Aula 13: avaliação da significância individual dos coeficientes de regressão</h3>
</div>

# Exemplo 1

<p style="text-align: justify; font-size:16px">
Considere o conjunto de dados <b>store.dat</b>, descrito no exemplo 2 da aula 2. Os objetivos são os seguintes: ajustar um modelo de regressão Poisson, no qual a variável resposta é o número de clientes e as preditoras são a distância ao concorrente mais próximo e a distância à loja e realizar as predições dos números médios de clientes  correspondentes às distâncias ao concorrente mais próximo e às distâncias à loja dadas na tabela abaixo.
    <table style="width:15%" align="center">
  <tr>
    <th style="text-align: center; vertical-align: middle;">distConc</th>
    <th style="text-align: center; vertical-align: middle;">distLoja</th>
  </tr>
  <tr>
    <td style="text-align: center; vertical-align: middle;">1</td>
    <td style="text-align: center; vertical-align: middle;">5</td>
  </tr>
  <tr>
    <td style="text-align: center; vertical-align: middle;">3</td>
    <td style="text-align: center; vertical-align: middle;">7</td>
  </tr>
  <tr>
    <td style="text-align: center; vertical-align: middle;">5</td>
    <td style="text-align: center; vertical-align: middle;">10</td>
  </tr>
</table>
<p>

### 1.1 Leitura e visualização dos dados

In [1]:
dados<-read.table("/Dados/Google_Drive_UFS/Aulas/2020_2/MLG/Dados/store.dat",header=TRUE)
n<-dim(dados)[1] #Número de observações
head(dados) #Visualizando as primeiras observações

,numClientes,numDom,rendaMedia,idadeDom,distConc,distLoja
,<int>,<int>,<int>,<int>,<dbl>,<dbl>
1,9,606,41393,3,3.04,6.32
2,6,641,23635,18,1.95,8.89
3,28,505,55475,27,6.54,2.05
4,11,866,64646,31,1.67,5.81
5,4,599,31972,7,0.72,8.11
6,4,520,41755,23,2.24,6.81


In [2]:
numClientes<-dados[,"numClientes"]
distConc<-dados[,"distConc"]
distLoja<-dados[,"distLoja"]

### 1.2 Formulação do modelo

<p style="text-align: justify; font-size:15px">
    <ol style="font-size:16px">
        <li>$numClientes\sim Poisson(\mu)$;</li>
        <li>$\eta=\beta_0+\beta_1\cdot distConc+\beta_2\cdot distLoja$;</li>
        <li>$\ln(\mu)=\eta$.</li>
    </ol>
</p>

### 1.3 Estimação dos parâmetros por máxima verossimilhança usando a função ``glm``

<p style="text-align: justify;font-size: 16px">Estimação dos coeficientes de regressão.</p>

In [3]:
aj<-glm(formula=numClientes~distConc+distLoja,family=poisson(link="log"),data=dados)
beta<-aj$coefficients

<p style="text-align: justify;font-size: 16px">Visualizando as estimativas dos coeficientes de regressão.</p>

In [4]:
print(beta)

(Intercept)    distConc    distLoja 
  2.5951177   0.1516029  -0.1090725 


### 1.4 Realizando as predições e calculando os intervalos de predição manualmente

<p style="text-align: justify; font-size:16px">Gerando o vetor dos valores observados da resposta e a matriz design.</p>

In [5]:
y<-numClientes
X<-cbind(distConc,distLoja)
X<-cbind(1,X)
colnames(X)<-c("","distConc","distLoja")
head(X)

,distConc,distLoja
1,3.04,6.32
1,1.95,8.89
1,6.54,2.05
1,1.67,5.81
1,0.72,8.11
1,2.24,6.81


<p style="text-align: justify; font-size:16px">No caso da distribuição Poisson, com ligação canônica, tem-se $\mu=g^{-1}(\eta)=e^{\eta}$. Logo, o valor de $\mu$ predito pelo modelo é 
    $$\hat\mu=g^{-1}(\hat\eta)=e^{\hat\eta}.$$
O intervalo de predição é
    $$\left(e^{\hat\eta-z_{\left(1-\frac{\alpha}{2}\right)}\sqrt{\hat{\sigma}^2_{\hat\eta}}};\ e^{\hat\eta+z_{\left(1-\frac{\alpha}{2}\right)}\sqrt{\hat{\sigma}^2_{\hat\eta}}}\right),$$
onde $\hat{\sigma}^2_{\hat\eta}=\boldsymbol{x}^T\boldsymbol{K}^{\ \ \ -1}_{\boldsymbol{\widehat\beta}\boldsymbol{\widehat\beta}}\boldsymbol{x},\ $ com $\ \boldsymbol{K}^{\ \ \ -1}_{\boldsymbol{\widehat\beta}\boldsymbol{\widehat\beta}}=(X^T{\widehat{\boldsymbol{V}}}X)^{-1},\ $ $\widehat{\boldsymbol{V}}=\hbox{diag}\{\hat\mu_1,\cdots\hat\mu_n\}\ $ e $\ \hat\mu_i=e^{\hat\eta_i}$, $\ \forall\ i\in\{1,\cdots,n\}$ (ver exemplo 2 da aula 5). 
</p>

<p style="text-align: justify;font-size: 16px">Calculando e visualizando a matriz $\boldsymbol{K}^{\ \ \ -1}_{\boldsymbol{\widehat\beta}\boldsymbol{\widehat\beta}}$.</p>

In [6]:
eta<-as.numeric(X%*%beta) #Estimativa de máxima verossimilhança do preditor linear
mu<-exp(eta) #Estimativa de máxima verossimilhança das médias
V<-diag(mu) #Calculando a estimativa da matriz V das funções de variância
Kbb<-t(X)%*%V%*%X #Calcuando Kbb
KbbInv<-solve(Kbb) #Calcuando a inversa de Kbb
betaVar<-diag(KbbInv) #Recolhendo a diagonal da inversa de Kbb, cujos valores correspondem às variâncias

<p style="text-align: justify;font-size: 16px">Construindo a matriz contendo os valores das variáveis preditoras para os quais deseja-se predizer os valores de $\mu$.</p>

In [7]:
wDistConc<-c(1,3,5)
wDistLoja<-c(5,7,10)
W<-cbind(wDistConc,wDistLoja)
W<-cbind(1,W)
head(W)

,wDistConc,wDistLoja
1,1,5
1,3,7
1,5,10


<p style="text-align: justify;font-size: 16px">Calculando e visualizando o valores de preditos de $\mu$.</p>

In [8]:
etaEst<-W%*%beta
muEst<-exp(etaEst)
yPred<-data.frame(distConc=wDistConc,distLoja=wDistLoja,mu=muEst)
yPred

distConc,distLoja,mu
<dbl>,<dbl>,<dbl>
1,5,9.037280
3,7,9.839640
5,10,9.606188


<p style="text-align: justify;font-size: 16px">Calculando e visualizando os intervalos de predição de $\mu$, com 95% de confiança.
</p>

In [9]:
alpha=0.05 #Valor de alpha correspondente ao intervalo com nível de confiança 1-alpha
z<-qnorm(1-alpha/2) #Quantil de ordem 1-alpha/2 da distribuição N(0,1)
covEta<-W%*%KbbInv%*%t(W) #Calculando a estimativa matriz de covariância do estimador de eta
sigma2Eta<-diag(covEta) #Extraindo a estimativa da variância do estimador de eta
ICEta<-cbind(etaEst-z*sqrt(sigma2Eta),etaEst+z*sqrt(sigma2Eta)) #Calculando os intervalos de confiança de eta
ICMu<-cbind(exp(ICEta)) #Calculando os intervalos de predição
print(ICMu)

         [,1]     [,2]
[1,] 7.680679 10.63349
[2,] 9.246062 10.47132
[3,] 7.954286 11.60115


### 1.5 Calculando as predições utilizando a função ``predict``

<p style="text-align: justify;font-size: 16px">A função <font face="'Courier New'">predict</font> calcula as predições de $\mu$. Os três principais argumentos da função são os seguintes:</p>
    <ol style="text-align: justify; font-size:16px">
        <li><font face="'Courier New'">object</font>: modelo ajustado. Saída da função <font face="'Courier New'">glm</font>.</li>
        <li><font face="'Courier New'">newdata</font>: variável do tipo <font face="'Courier New'">data.frame</font> contendo os valores das variáveis preditoras para os quais se deseja predizer $\mu$.</li>
    <li><font face="'Courier New'">type</font>: indica se o valor a ser calculado é o valor do preditor linear $\eta$ (<font face="'Courier New'">type="link"</font>) ou o valor da média $\mu$ (<font face="'Courier New'">type="response"</font>).</li>
    </ol>
</p>
<p style="text-align: justify;font-size: 16px">Os nomes das colunas de <font face="'Courier New'">newdata</font> devem ser iguais aos nomes das variáveis preditoras. </p>

<p style="text-align: justify;font-size: 16px">Construindo o <font face="'Courier New'">data.frame</font> contendo os valores das variáveis preditoras para os quais se deseja predizer $\mu$.
</p>

In [10]:
W<-data.frame(distConc=wDistConc,distLoja=wDistLoja)
W

distConc,distLoja
<dbl>,<dbl>
1,5
3,7
5,10


<p style="text-align: justify;font-size: 16px">Realizando a predição por meio da função <font face="'Courier New'">predict</font>.
</p>

In [11]:
muEst<-predict(object=aj,newdata=W,type="response")
yPred<-data.frame(distConc=wDistConc,distLoja=wDistLoja,mu=muEst)
yPred

,distConc,distLoja,mu
,<dbl>,<dbl>,<dbl>
1,1,5,9.037280
2,3,7,9.839640
3,5,10,9.606188


### 1.6 Calculando os intervalos de predição, com 95% de confiança, utilizando a função ``add_ci``

<p style="text-align: justify;font-size: 16px">Carregando os pacotes necessários.</p>

In [12]:
library(tibble)
library(ciTools)

ciTools version 0.5.1 (C) Institute for Defense Analyses



<p style="text-align: justify;font-size: 16px">A função <font face="'Courier New'">add_ci</font> calcula os intervalos de predição. Os três principais argumentos dessa função são:</p>
   <ol style="text-align: justify; font-size:16px">
        <li><font face="'Courier New'">tb</font>: variável do tipo <font face="'Courier New'">tibble</font> contendo os valores das variáveis preditoras para os quais se deseja predizer $\mu$.</li>
        <li><font face="'Courier New'">fit</font>: modelo ajustado. Saída da função <font face="'Courier New'">glm</font>.</li>
    <li><font face="'Courier New'">alpha</font>: nível de confiança. O nível de confiança do intervalo de predição é <font face="'Courier New'">1-alpha</font>.</li>
    </ol>
</p>
<p style="text-align: justify;font-size: 16px">O tipo <font face="'Courier New'">tibble</font>  é semelhante ao ao tipo <font face="'Courier New'">data.frame</font>, porém com algumas diferenças. Para mais informações, consulte a ajuda da função, por meio do comando <font face="'Courier New'">?tibble</font>.</p>

<p style="text-align: justify;font-size: 16px">Construindo o <font face="'Courier New'">tibble</font> contendo os valores das variáveis preditoras para os quais se deseja predizer $\mu$. Os nomes das colunas devem ser iguais aos das variáveis.
</p>

In [13]:
W<-tibble(distConc=wDistConc,distLoja=wDistLoja)
W

distConc,distLoja
<dbl>,<dbl>
1,5
3,7
5,10


<p style="text-align: justify;font-size: 16px">Calculando os intervalos de predição utilizando a função <font face="'Courier New'">add_ci</font>.</p>

In [14]:
add_ci(tb=W,fit=aj)

Warning message:
“`as_data_frame()` is deprecated as of tibble 2.0.0.
Please use `as_tibble()` instead.
The signature and semantics have changed, see `?as_tibble`.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”


distConc,distLoja,pred,LCB0.025,UCB0.975
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,5,9.037280,7.680683,10.63349
3,7,9.839640,9.246064,10.47132
5,10,9.606188,7.954289,11.60114
